#  Ψαξίματα

* Με προσαρμοσμένο υλικό από την τεκμηρίωση του scikit-learn.

---

> Πάνος Λουρίδας, Αναπληρωτής Καθηγητής <br />
> Τμήμα Διοικητικής Επιστήμης και Τεχνολογίας <br />
> Οικονομικό Πανεπιστήμιο Αθηνών <br />
> louridas@aueb.gr

In [1]:
from sklearn import datasets
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

import pandas as pd
import numpy as np
from scipy import stats

* Ας ξαναπιάσουμε το πρόβλημα με τις βιοψίες και τον καρκίνο του μαστού.

In [2]:
bc = datasets.load_breast_cancer(as_frame=True)
bc.data

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


* Θέλουμε να δοκιμάσουμε Μηχανές Διανυσμάτων Στήριξης, αλλά δεν έχουμε ιδέα για τις υπερπαραμέτρους που θα πρέπει να χρησιμοποιήσουμε.

* Ορίζουμε λοιπόν μία δομή με τις τιμές που θέλουμε να δοκιμάσουμε για κάθε υπερπαράμετρο.

In [3]:
param_grid = [
    {
        "kernel": ["rbf"], 
        "gamma": [1e-2, 1e-3, 1e-4], 
        "C": [1, 10, 100, 1000],
        "decision_function_shape": ["ovo", "ovr"]
    },
    {
        "kernel": ["linear"], 
        "C": [1, 10, 100, 1000],
        "decision_function_shape": ["ovo", "ovr"]
    },
]

* Στη συνέχεια, χρησιμοποιούμε την κλάση `GridSearchCV`, η οποία θα δοκιμάσει όλους τους δυνατούς συνδυασμούς των παραμέτρων και θα εντοπίσει το καλύτερο μοντέλο.

* H `GridSearchCV`, όπως λέει και το όνομά της πραγματοποιεί και $k$-fold validation, όπου $k = 5$ αν δεν ορίσουμε εμείς κάτι άλλο.

* Επίσης, όπως και άλλες μέθοδοι του scikit-learn που επιδέχονται παραλληλοποίηση, με την παράμετρο `n_jobs` ζητούμε τον αριθμό των παράλληλων διεργασιών που θέλουμε να εκτελεστούν (-1 σημαίνει όσες μπορεί).

In [4]:
clf = SVC()
grid_search = GridSearchCV(clf, param_grid=param_grid, n_jobs=-1)
grid_search.fit(bc.data, bc.target)

,estimator,SVC()
,param_grid,"[{'C': [1, 10, ...], 'decision_function_shape': ['ovo', 'ovr'], 'gamma': [0.01, 0.001, ...], 'kernel': ['rbf']}, {'C': [1, 10, ...], 'decision_function_shape': ['ovo', 'ovr'], 'kernel': ['linear']}]"
,scoring,None
,n_jobs,-1
,refit,True
,cv,None
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,C,100


* Για να δούμε τις βέλτιστες τιμές των υπερπαραμέτρων, μπορούμε να χρησιμοποιήσουμε την ιδιότητα `best_params_`.

In [5]:
grid_search.best_params_

{'C': 100, 'decision_function_shape': 'ovo', 'kernel': 'linear'}

* Αν θέλουμε να πάρουμε το καλύτερο μοντέλο για να το χρησιμοποιήσουμε, αυτό είναι διαθέσιμο από την ιδιότητα `best_estimator_`.

In [6]:
grid_search.best_estimator_

,C,100
,kernel,'linear'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


* Αν θέλουμε να δούμε όλα τα αποτελέσματα, μπορούμε να τα πάρουμε με την ιδιότητα `cv_results_`.

In [7]:
grid_search_results = pd.DataFrame(grid_search.cv_results_)
print(grid_search_results.shape)
grid_search_results

(32, 17)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_decision_function_shape,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.029573,0.006970,0.020619,0.007549,1,ovo,0.0100,rbf,"{'C': 1, 'decision_function_shape': 'ovo', 'ga...",0.622807,0.622807,0.631579,0.631579,0.619469,0.625648,0.004993,31
1,0.027416,0.008740,0.017187,0.007465,1,ovo,0.0010,rbf,"{'C': 1, 'decision_function_shape': 'ovo', 'ga...",0.929825,0.921053,0.921053,0.947368,0.893805,0.922621,0.017318,17
2,0.009626,0.000807,0.005615,0.000588,1,ovo,0.0001,rbf,"{'C': 1, 'decision_function_shape': 'ovo', 'ga...",0.903509,0.938596,0.938596,0.956140,0.938053,0.934979,0.017169,11
3,0.026157,0.004497,0.013619,0.002481,1,ovr,0.0100,rbf,"{'C': 1, 'decision_function_shape': 'ovr', 'ga...",0.622807,0.622807,0.631579,0.631579,0.619469,0.625648,0.004993,31
4,0.018723,0.002273,0.009818,0.001934,1,ovr,0.0010,rbf,"{'C': 1, 'decision_function_shape': 'ovr', 'ga...",0.929825,0.921053,0.921053,0.947368,0.893805,0.922621,0.017318,17
5,0.010412,0.001287,0.006715,0.000818,1,ovr,0.0001,rbf,"{'C': 1, 'decision_function_shape': 'ovr', 'ga...",0.903509,0.938596,0.938596,0.956140,0.938053,0.934979,0.017169,11
6,0.026836,0.003155,0.013721,0.001917,10,ovo,0.0100,rbf,"{'C': 10, 'decision_function_shape': 'ovo', 'g...",0.622807,0.622807,0.631579,0.640351,0.619469,0.627403,0.007619,25
7,0.024600,0.004335,0.010688,0.001538,10,ovo,0.0010,rbf,"{'C': 10, 'decision_function_shape': 'ovo', 'g...",0.894737,0.903509,0.921053,0.938596,0.884956,0.908570,0.019142,19
8,0.014939,0.003096,0.006227,0.001180,10,ovo,0.0001,rbf,"{'C': 10, 'decision_function_shape': 'ovo', 'g...",0.894737,0.938596,0.947368,0.956140,0.920354,0.931439,0.021841,13
9,0.029542,0.005813,0.018558,0.010398,10,ovr,0.0100,rbf,"{'C': 10, 'decision_function_shape': 'ovr', 'g...",0.622807,0.622807,0.631579,0.640351,0.619469,0.627403,0.007619,25


* Ίσως είναι καλύτερα να τα ταξινομήσουμε σε φθίνουσα σειρά επίδοσης.

In [8]:
grid_search_results.sort_values(by='rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_decision_function_shape,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
29,13.844450,4.516383,0.002300,0.000401,100,ovr,NaN,linear,"{'C': 100, 'decision_function_shape': 'ovr', '...",0.938596,0.947368,0.973684,0.973684,0.982301,0.963127,0.016974,1
28,13.300475,3.190980,0.003000,0.000001,100,ovo,NaN,linear,"{'C': 100, 'decision_function_shape': 'ovo', '...",0.938596,0.947368,0.973684,0.973684,0.982301,0.963127,0.016974,1
31,12.659875,9.633494,0.002815,0.000397,1000,ovr,NaN,linear,"{'C': 1000, 'decision_function_shape': 'ovr', ...",0.947368,0.947368,0.973684,0.929825,0.973451,0.954339,0.016957,3
30,12.476535,10.587343,0.002505,0.000639,1000,ovo,NaN,linear,"{'C': 1000, 'decision_function_shape': 'ovo', ...",0.947368,0.947368,0.973684,0.929825,0.973451,0.954339,0.016957,3
26,5.568925,1.123604,0.003543,0.001081,10,ovo,NaN,linear,"{'C': 10, 'decision_function_shape': 'ovo', 'k...",0.938596,0.938596,0.973684,0.947368,0.964602,0.952569,0.014200,5
27,5.736415,1.165088,0.002806,0.000511,10,ovr,NaN,linear,"{'C': 10, 'decision_function_shape': 'ovr', 'k...",0.938596,0.938596,0.973684,0.947368,0.964602,0.952569,0.014200,5
24,1.840538,0.713342,0.002894,0.000467,1,ovo,NaN,linear,"{'C': 1, 'decision_function_shape': 'ovo', 'ke...",0.947368,0.929825,0.973684,0.921053,0.955752,0.945536,0.018689,7
25,1.765152,0.699288,0.002963,0.000519,1,ovr,NaN,linear,"{'C': 1, 'decision_function_shape': 'ovr', 'ke...",0.947368,0.929825,0.973684,0.921053,0.955752,0.945536,0.018689,7
14,0.019520,0.008100,0.007163,0.001862,100,ovo,0.0001,rbf,"{'C': 100, 'decision_function_shape': 'ovo', '...",0.947368,0.938596,0.956140,0.938596,0.911504,0.938441,0.014957,9
17,0.014066,0.000923,0.005008,0.000637,100,ovr,0.0001,rbf,"{'C': 100, 'decision_function_shape': 'ovr', '...",0.947368,0.938596,0.956140,0.938596,0.911504,0.938441,0.014957,9


* Αν θέλουμε απευθείας την καλύτερη επίδοση:

In [9]:
print(grid_search.best_score_)

0.9631268436578171


* Όπως και προηγουμένως, μπορούμε να δοκιμάσουμε και με έναν `SGDClassifier`.

* Θα ξεκινήσουμε με μία διαφορετική προσέγγιση.

* Αντί στην αναζήτηση να εξαντλήσουμε όλους τους δυνατούς συνδυασμούς, θα κάνουμε τυχαία δειγματοληψία μεταξύ τους.

In [10]:
param_dist = {
    "average": [True, False],
    "l1_ratio": stats.uniform(0, 1),
    "alpha": stats.loguniform(1e-2, 1e0),
}

* Για να το κάνουμε αυτό, χρησιμοποιούμε την κλάση `RandomizedSearchCV` δίνοντας τον αριθμό των δειγμάτων που θέλουμε να δοκιμάσει από τον χώρο των δυνατών συνδυασμών των παραμέτρων.

In [11]:
clf = SGDClassifier()
random_search = RandomizedSearchCV(clf, param_distributions=param_dist, n_iter=15, n_jobs=-1)
random_search.fit(bc.data, bc.target)

,estimator,SGDClassifier()
,param_distributions,"{'alpha': <scipy.stats....002228B9C5910>, 'average': [True, False], 'l1_ratio': <scipy.stats....002228B704E30>}"
,n_iter,15
,scoring,None
,n_jobs,-1
,refit,True
,cv,None
,verbose,0
,pre_dispatch,'2*n_jobs'
,random_state,None
,error_score,nan


* Για να δούμε τι βρήκαμε:

In [12]:
random_search.best_params_

{'alpha': np.float64(0.07583406134859368),
 'average': False,
 'l1_ratio': np.float64(0.46386246406669807)}

* Και τον βέλτιστο εκτιμητή:

In [13]:
random_search.best_estimator_

,loss,'hinge'
,penalty,'l2'
,alpha,np.float64(0....3406134859368)
,l1_ratio,np.float64(0....6246406669807)
,fit_intercept,True
,max_iter,1000
,tol,0.001
,shuffle,True
,verbose,0
,epsilon,0.1
,n_jobs,None


* Και πόσο καλός είναι:

In [14]:
print(random_search.best_score_)

0.903369042074212


* Αν θέλουμε (και μπορούμε) να κάνουμε εξαντλητική αναζήτηση στις τιμές, βεβαίως προχωράμε όπως προηγουμένως:

In [15]:
param_grid = {
    "average": [True, False],
    "l1_ratio": np.linspace(0, 1, num=10),
    "alpha": np.power(10, np.arange(-2, 1, dtype=float)),
}

* Χρησιμοποιούμε πάλι την κλάση `GridSearchCV`.

In [16]:
clf = SGDClassifier()
grid_search = GridSearchCV(clf, param_grid=param_grid, n_jobs=-1)
grid_search.fit(bc.data, bc.target)

,estimator,SGDClassifier()
,param_grid,"{'alpha': array([0.01, 0.1 , 1. ]), 'average': [True, False], 'l1_ratio': array([0. ..., 1. ])}"
,scoring,None
,n_jobs,-1
,refit,True
,cv,None
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,loss,'hinge'


* Για να δούμε τι βρήκαμε:

In [17]:
grid_search.best_params_

{'alpha': np.float64(0.1),
 'average': False,
 'l1_ratio': np.float64(0.5555555555555556)}

* Και τον καλύτερο εκτιμητή:

In [18]:
grid_search.best_estimator_

,loss,'hinge'
,penalty,'l2'
,alpha,np.float64(0.1)
,l1_ratio,np.float64(0.5555555555555556)
,fit_intercept,True
,max_iter,1000
,tol,0.001
,shuffle,True
,verbose,0
,epsilon,0.1
,n_jobs,None


* Και την επίδοσή του:

In [19]:
print(grid_search.best_score_)

0.9138798323241734


* Αλλά δεν πρέπει να δοκιμάσουμε και κανονικοποίηση;

* Θα φτιάξουμε λοιπόν μια διοχέτευση.

In [20]:
clf = make_pipeline(StandardScaler(), SGDClassifier())

* Μπορούμε να ορίσουμε παραμέτρους προς αναζήτηση για κάθε ένα από τα βήματα μιας διοχέτευσης.

* Εδώ θα δώσουμε μόνο για το δεύτερο.

In [21]:
param_grid = {
    "sgdclassifier__average": [True, False],
    "sgdclassifier__l1_ratio": np.linspace(0, 1, num=10),
    "sgdclassifier__alpha": np.power(10, np.arange(-2, 1, dtype=float)),
}

* Οπότε προχωράμε στην αναζήτηση...

In [22]:
grid_search = GridSearchCV(clf, param_grid=param_grid, n_jobs=-1)
grid_search.fit(bc.data, bc.target)

,estimator,Pipeline(step...lassifier())])
,param_grid,"{'sgdclassifier__alpha': array([0.01, 0.1 , 1. ]), 'sgdclassifier__average': [True, False], 'sgdclassifier__l1_ratio': array([0. ..., 1. ])}"
,scoring,None
,n_jobs,-1
,refit,True
,cv,None
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,copy,True


* Και να τι βγάλαμε:

In [23]:
grid_search.best_params_

{'sgdclassifier__alpha': np.float64(0.01),
 'sgdclassifier__average': True,
 'sgdclassifier__l1_ratio': np.float64(0.4444444444444444)}

* Ο καλύτερος εκτιμητής:

In [24]:
grid_search.best_estimator_

,steps,"[('standardscaler', ...), ('sgdclassifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,copy,True
,with_mean,True
,with_std,True
,loss,'hinge'
,penalty,'l2'
,alpha,np.float64(0.01)
,l1_ratio,np.float64(0.4444444444444444)


* Και η βέλτιστη επίδοση:

In [25]:
print(grid_search.best_score_)

0.9789163173420278


* Μην ξεχνάμε: ο υπολογιστής υπάρχει για να μπορούμε εμείς να ξεφορτωνόμαστε επάνω του τη βαρετή δουλειά.

* Αρκεί βέβαια να μας παίρνει να το κάνουμε...

* Και δεν σημαίνει ότι δεν πρέπει να αφιερώσουμε χρόνο στην τεκμηρίωση ώστε να ξέρουμε πραγματικά τι είναι οι υπερπαράμετροι που χρησιμοποιούμε.